# Immigration Data Lake
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:3.0.0-s_2.12")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
# Read in the data here
fname = 'data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [4]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [6]:
# save to csv in data folder
df.to_csv('data/i94_apr16_sub.csv')

In [10]:
# save a sample of the csv to the data folder
df.sample(1000).to_csv('data/i94_apr16_sub_sample.csv')

In [1]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:3.0.0-s_2.12")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [ ]:
#write to parquet
df_spark.write.parquet("data/sas_data")
df_spark=spark.read.parquet("data/sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Immigration Data

In [23]:
# read immigration data
imm_df = spark.read.format('com.github.saurfang.sas.spark').load('data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [5]:
# immigration data schema
imm_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [6]:
# | I94RES | 3 digit code for country of residence |
imm_df.groupBy('I94RES').count().show()

+------+------+
|I94RES| count|
+------+------+
| 692.0| 42495|
| 299.0|   792|
| 576.0| 13992|
| 735.0|   423|
| 206.0|  8505|
| 524.0|   519|
| 389.0|    99|
| 390.0|    76|
| 249.0|  1310|
| 329.0|   153|
| 112.0|156613|
| 154.0|   616|
| 521.0|    19|
| 124.0| 20245|
| 438.0|112407|
| 348.0|   137|
| 410.0|    42|
| 253.0|  2646|
| 317.0|     9|
| 128.0|    24|
+------+------+
only showing top 20 rows



In [7]:
# | I94ADDR | state of arrival |
imm_df.groupBy('I94ADDR').count().show()

+-------+-----+
|I94ADDR|count|
+-------+-----+
|     .N|    4|
|     RG|    4|
|     YH|    1|
|     RF|    1|
|     FT|    2|
|     CI|   13|
|     TC|    1|
|     SC| 9811|
|     AZ|20218|
|     FI|    7|
|     IC|    3|
|     PU|   13|
|     UA|   10|
|     EA|    2|
|     NS|   28|
|     KI|   25|
|     RO|    1|
|     PI|    6|
|     SL|    6|
|     LA|22655|
+-------+-----+
only showing top 20 rows



In [8]:
# | I94VISA | visa codes collapsed into three categories: 1 = Business; 2 = Pleasure; 3 = Student |
imm_df.groupBy('I94VISA').count().show()

+-------+-------+
|I94VISA|  count|
+-------+-------+
|    1.0| 522079|
|    3.0|  43366|
|    2.0|2530868|
+-------+-------+



In [9]:
imm_df.groupBy('I94PORT').count().show()

+-------+-----+
|I94PORT|count|
+-------+-----+
|    FMY|17514|
|    BGM|  128|
|    HEL|    2|
|    DNS|   35|
|    MOR|   14|
|    FOK|   14|
|    HVR|   45|
|    SNA| 7066|
|    PTK|   12|
|    SPM|16973|
|    CLG| 3191|
|    OPF|  909|
|    DLB|   12|
|    ABS|    4|
|    NAS|13032|
|    MYR|    5|
|    PVD|  352|
|    OAK| 3501|
|    FAR|    5|
|    OTT|  663|
+-------+-----+
only showing top 20 rows



In [10]:
imm_df.select('I94VISA').show()

+-------+
|I94VISA|
+-------+
|    2.0|
|    3.0|
|    2.0|
|    2.0|
|    2.0|
|    1.0|
|    2.0|
|    2.0|
|    2.0|
|    1.0|
|    2.0|
|    2.0|
|    1.0|
|    1.0|
|    2.0|
|    2.0|
|    2.0|
|    2.0|
|    2.0|
|    2.0|
+-------+
only showing top 20 rows



In [30]:
imm_df.groupBy('I94YR').count().show()

+------+-------+
| I94YR|  count|
+------+-------+
|2016.0|3096313|
+------+-------+



In [31]:
gr = imm_df.groupBy("INSNUM").agg(countDistinct("GENDER"))
gr.show()

+------+-------------+
|INSNUM|count(GENDER)|
+------+-------------+
|  4032|            1|
|  5325|            3|
| 39645|            1|
|  4821|            1|
| 39590|            1|
|  3414|            2|
|  7252|            2|
| 39458|            1|
|   691|            1|
|  3959|            1|
|  3441|            3|
|  3517|            4|
|  3281|            3|
| 38900|            1|
|  5067|            1|
|  3650|            2|
|  3826|            1|
|  5023|            2|
|  1669|            3|
|  5149|            3|
+------+-------------+
only showing top 20 rows



In [32]:
gr.groupBy('count(GENDER)').count().show()

+-------------+-----+
|count(GENDER)|count|
+-------------+-----+
|            1|  911|
|            3|  266|
|            2|  654|
|            4|   83|
+-------------+-----+



In [12]:
# Wrong date times, because its a SAS numeric time field
from pyspark.sql.functions import col , column
imm_df.withColumn("ts", col("ARRDATE").cast("timestamp")).select('ts').show()

+-------------------+
|                 ts|
+-------------------+
|1970-01-01 05:42:53|
|1970-01-01 05:42:31|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
|1970-01-01 05:42:25|
+-------------------+
only showing top 20 rows



#### Weather Data

In [13]:
weather_df = spark.read.options(header='true', inferSchema='true').csv('data/GlobalLandTemperaturesByState.csv')

In [14]:
weather_df.printSchema()

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)



In [15]:
weather_df.show(10)

+----------+------------------+-----------------------------+-----+-------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|State|Country|
+----------+------------------+-----------------------------+-----+-------+
|1855-05-01|            25.544|                        1.171| Acre| Brazil|
|1855-06-01|            24.228|                        1.103| Acre| Brazil|
|1855-07-01|            24.371|                        1.044| Acre| Brazil|
|1855-08-01|            25.427|                        1.073| Acre| Brazil|
|1855-09-01|            25.675|                        1.014| Acre| Brazil|
|1855-10-01|25.441999999999997|                        1.179| Acre| Brazil|
|1855-11-01|              25.4|                        1.064| Acre| Brazil|
|1855-12-01|              24.1|           1.7180000000000002| Acre| Brazil|
|1856-01-01|            25.814|                        1.159| Acre| Brazil|
|1856-02-01|            24.658|                        1.147| Acre| Brazil|
+----------+

In [16]:
weather_df.groupBy('State').count().show()

+--------------+-----+
|         State|count|
+--------------+-----+
|          Utah| 2298|
|     Volgograd| 3239|
|       Bryansk| 3239|
|        Hawaii| 1569|
|      Voronezh| 3239|
|      Manitoba| 2941|
|      Kostroma| 3239|
|      Nagaland| 2371|
|     Karnataka| 2613|
|      Belgorod| 3239|
|     Guangdong| 2085|
|     Minnesota| 3239|
|          Omsk| 2421|
|Santa Catarina| 2181|
|         Kursk| 3239|
|          Ohio| 3239|
|         Hunan| 2082|
|        Kerala| 2613|
|        Shanxi| 2318|
|        Kaluga| 3239|
+--------------+-----+
only showing top 20 rows



#### Demographics data

In [49]:
# demo
demo_df = spark.read.options(header='true', inferSchema='true', delimiter=';').csv('data/us-cities-demographics.csv')

In [50]:
demo_df.limit(10).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Wichita,Kansas,34.6,192354,197601,389955,23978.0,40270.0,2.56,KS,American Indian and Alaska Native,8791
1,Allen,Pennsylvania,33.5,60626,59581,120207,5691.0,19652.0,2.67,PA,Black or African-American,22304
2,Danbury,Connecticut,37.3,43435,41227,84662,3752.0,25675.0,2.74,CT,Black or African-American,8454
3,Nashville,Tennessee,34.1,314231,340365,654596,27942.0,88193.0,2.39,TN,Hispanic or Latino,67526
4,Stamford,Connecticut,35.4,64941,63936,128877,2269.0,44003.0,2.70,CT,Asian,11013
5,San Juan,Puerto Rico,41.4,155408,186829,342237,NaN,NaN,NaN,PR,Hispanic or Latino,335559
6,Provo,Utah,23.6,56231,59027,115258,2177.0,10925.0,3.28,UT,White,108471
7,San Marcos,California,35.4,45246,47688,92934,5189.0,21558.0,3.13,CA,Black or African-American,4447
8,Escondido,California,33.3,76551,74907,151458,8110.0,46298.0,3.27,CA,American Indian and Alaska Native,3151
9,Caguas,Puerto Rico,40.4,34743,42265,77008,NaN,NaN,NaN,PR,Hispanic or Latino,76349


#### Airport data

In [55]:
# airport
airport_df = spark.read.options(header='true', inferSchema='true').csv('data/airport-codes_csv.csv')

In [56]:
airport_df.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [69]:
airport_df.groupBy('iso_country').count().show()

+-----------+-----+
|iso_country|count|
+-----------+-----+
|         DZ|   61|
|         LT|   55|
|         MM|   75|
|         CI|   26|
|         TC|   10|
|         AZ|   35|
|         FI|  111|
|         SC|   18|
|         PM|    2|
|         UA|  191|
|         ZM|  103|
|         KI|   22|
|         RO|   59|
|         SL|   12|
|         SB|   38|
|         NL|   90|
|         LA|   20|
|         BS|   66|
|         BW|  127|
|         MN|   29|
+-----------+-----+
only showing top 20 rows



In [65]:
airport_df.filter((airport_df["ident"] == "") | airport_df["ident"].isNull()).count()

0

In [66]:
airport_df.groupBy('ident').count().count()

55075

In [67]:
airport_df.count()

55075

#### Cleaning Steps
Document steps necessary to clean the data

#### Immigration data cleaning

In [17]:
imm_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [ ]:
# rename cicid to id
# rename i94yr to year
# rename i94mon to month
# read mapping for code to country of origin, and map the I94CIT var to the actual country of origin (do later)

In [24]:
imm_fact = imm_df.select(col("cicid").alias("id"),
                          col("i94yr").alias("year"),
                          col("i94mon").alias("month"),
                          col("I94CIT").alias("country_of_origin"),
                          col("I94ADDR").alias("state_of_arrival"))

In [25]:
imm_fact.show()

+----+------+-----+-----------------+----------------+
|  id|  year|month|country_of_origin|state_of_arrival|
+----+------+-----+-----------------+----------------+
| 6.0|2016.0|  4.0|            692.0|            null|
| 7.0|2016.0|  4.0|            254.0|              AL|
|15.0|2016.0|  4.0|            101.0|              MI|
|16.0|2016.0|  4.0|            101.0|              MA|
|17.0|2016.0|  4.0|            101.0|              MA|
|18.0|2016.0|  4.0|            101.0|              MI|
|19.0|2016.0|  4.0|            101.0|              NJ|
|20.0|2016.0|  4.0|            101.0|              NJ|
|21.0|2016.0|  4.0|            101.0|              NY|
|22.0|2016.0|  4.0|            101.0|              NY|
|23.0|2016.0|  4.0|            101.0|              NY|
|24.0|2016.0|  4.0|            101.0|              MO|
|27.0|2016.0|  4.0|            101.0|              MA|
|28.0|2016.0|  4.0|            101.0|              MA|
|29.0|2016.0|  4.0|            101.0|              MA|
|30.0|2016

In [33]:
person_dim = imm_df.select(col("cicid").alias("id"),
                          col("GENDER").alias("gender"),
                          col("I94BIR").alias("I94BIR"))

In [34]:
person_dim.show()

+----+------+------+
|  id|gender|I94BIR|
+----+------+------+
| 6.0|  null|  37.0|
| 7.0|     M|  25.0|
|15.0|     M|  55.0|
|16.0|  null|  28.0|
|17.0|  null|   4.0|
|18.0|  null|  57.0|
|19.0|  null|  63.0|
|20.0|  null|  57.0|
|21.0|  null|  46.0|
|22.0|  null|  48.0|
|23.0|  null|  52.0|
|24.0|  null|  33.0|
|27.0|     M|  58.0|
|28.0|     F|  56.0|
|29.0|     M|  62.0|
|30.0|     M|  49.0|
|31.0|     M|  43.0|
|33.0|     F|  53.0|
|34.0|     M|  48.0|
|35.0|     F|  74.0|
+----+------+------+
only showing top 20 rows



In [41]:
demo_df.show(5)

+---------+------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|     City|       State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+---------+------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|  Wichita|      Kansas|      34.6|         192354|           197601|          389955|             23978|       40270|                  2.56|        KS|American Indian a...| 8791|
|    Allen|Pennsylvania|      33.5|          60626|            59581|          120207|              5691|       19652|                  2.67|        PA|Black or African-...|22304|
|  Danbury| Connecticut|      37.3|          43435|            41227|           84662|              

In [51]:
demo_df = demo_df.withColumn("id", monotonically_increasing_id())

In [52]:
demo_df.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)
 |-- id: long (nullable = false)



In [53]:
demographics_dim = demo_df.select(col("id"),
                                  col("State").alias("state"),
                                  col("MEDIAN AGE").alias("median_age"),
                                  col("Total Population").alias("total_population"))

In [54]:
demographics_dim.show(5)

+---+------------+----------+----------------+
| id|       state|median_age|total_population|
+---+------------+----------+----------------+
|  0|      Kansas|      34.6|          389955|
|  1|Pennsylvania|      33.5|          120207|
|  2| Connecticut|      37.3|           84662|
|  3|   Tennessee|      34.1|          654596|
|  4| Connecticut|      35.4|          128877|
+---+------------+----------+----------------+
only showing top 5 rows



In [57]:
airport_df.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [68]:
airport_df.limit(10).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,id
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125",0
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022",1
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968",2
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172",3
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087",4
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028",5
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484",6
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995",7
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188",8
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736",9


In [ ]:
airports_dim = airport_df.select(col("ident").alias("id"),
                                  col("iso_country").alias("country"),
                                  col("MEDIAN AGE").alias("median_age"),
                                  col("Total Population").alias("total_population"))

### Create country code mapping table using the I94 description file

In [59]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/i94_country_code_mapping.csv", header=None)

In [60]:
df = df.replace(np.nan, '', regex=True)
cols = [1,2,3,4,5,6,7,8,9,10]
df['country'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df = df[[0, 'country']]
df.columns = ['code', 'country']
df = df[df['code'] != '']
df['code'] = df['code'].astype(int)
df.to_csv('data/code_to_country_mapping.csv', index=False)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [1]:
# Imports
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as F
from pyspark.sql.functions import col, create_map, lit, expr, to_date
from itertools import chain

In [5]:
spark = SparkSession.builder.\
config("spark.jars.packages")\
.enableHiveSupport().getOrCreate()

Exception: Java gateway process exited before sending its port number

In [4]:
# Read all tables
immigration_staging = spark.read.options(header='true', inferSchema='true').csv('data/i94_apr16_sub_sample.csv')
country_code_mapping_staging = spark.read.options(header='true', inferSchema='true').csv('data/code_to_country_mapping.csv')
demographics_staging = spark.read.options(header='true', inferSchema='true', delimiter=';').csv('data/us-cities-demographics.csv')
weather_staging = spark.read.options(header='true', inferSchema='true').csv('data/GlobalLandTemperaturesByState.csv')
airport_staging = spark.read.options(header='true', inferSchema='true').csv('data/airport-codes_csv.csv')

NameError: name 'spark' is not defined

In [ ]:
transportation_mapping = {1: 'air', 2: 'sea', 3: 'land', 4: 'not reported'}
transportation_mapping_expr = create_map([lit(x) for x in chain(*transportation_mapping.items())])

state_mapping = {'AL':'ALABAMA',
                    'AK':'ALASKA',
                    'AZ':'ARIZONA',
                    'AR':'ARKANSAS',
                    'CA':'CALIFORNIA',
                    'CO':'COLORADO',
                    'CT':'CONNECTICUT',
                    'DE':'DELAWARE',
                    'DC':'DIST. OF COLUMBIA',
                    'FL':'FLORIDA',
                    'GA':'GEORGIA',
                    'GU':'GUAM',
                    'HI':'HAWAII',
                    'ID':'IDAHO',
                    'IL':'ILLINOIS',
                    'IN':'INDIANA',
                    'IA':'IOWA',
                    'KS':'KANSAS',
                    'KY':'KENTUCKY',
                    'LA':'LOUISIANA',
                    'ME':'MAINE',
                    'MD':'MARYLAND',
                    'MA':'MASSACHUSETTS',
                    'MI':'MICHIGAN',
                    'MN':'MINNESOTA',
                    'MS':'MISSISSIPPI',
                    'MO':'MISSOURI',
                    'MT':'MONTANA',
                    'NC':'N. CAROLINA',
                    'ND':'N. DAKOTA',
                    'NE':'NEBRASKA',
                    'NV':'NEVADA',
                    'NH':'NEW HAMPSHIRE',
                    'NJ':'NEW JERSEY',
                    'NM':'NEW MEXICO',
                    'NY':'NEW YORK',
                    'OH':'OHIO',
                    'OK':'OKLAHOMA',
                    'OR':'OREGON',
                    'PA':'PENNSYLVANIA',
                    'PR':'PUERTO RICO',
                    'RI':'RHODE ISLAND',
                    'SC':'S. CAROLINA',
                    'SD':'S. DAKOTA',
                    'TN':'TENNESSEE',
                    'TX':'TEXAS',
                    'UT':'UTAH',
                    'VT':'VERMONT',
                    'VI':'VIRGIN ISLANDS',
                    'VA':'VIRGINIA',
                    'WV':'W. VIRGINIA',
                    'WA':'WASHINGTON',
                    'WI':'WISCONSON',
                    'WY':'WYOMING' ,
                    '99':'All Other Codes'}
state_mapping = dict((k, v.title()) for k, v in state_mapping.items())
state_mapping_expr = create_map([lit(x) for x in chain(*state_mapping.items())])

visa_mapping = {1: 'business', 2: 'pleasure', 3: 'student'}
visa_mapping_expr = create_map([lit(x) for x in chain(*visa_mapping.items())])

immigration_fact =  immigration_staging.select(col("cicid").alias("id").cast("int"),
                    col("I94YR").alias("year").cast("int"),
                    col("I94MON").alias("month").cast("int"),
                    col("I94PORT").alias("port_of_entry"),
                    col("ARRDATE").alias("arrival_date").cast("int"),
                    col("I94MODE").alias("mode_of_transportation").cast("int"),
                    col("I94ADDR").alias("state_of_arrival_code"),
                    col("DEPDATE").alias("departure_date").cast("int"),
                    col("I94VISA").alias("visa_type").cast("int"),) \
.withColumn("mode_of_transportation", transportation_mapping_expr[col("mode_of_transportation")]) \
.withColumn("state_of_arrival", state_mapping_expr[col("state_of_arrival_code")]) \
.withColumn("sas_date", to_date(lit("01/01/1960"), "MM/dd/yyyy")) \
.withColumn("arrival_date", expr("date_add(sas_date, arrival_date)")) \
.withColumn("departure_date", expr("date_add(sas_date, departure_date)")) \
.withColumn("visa_type", visa_mapping_expr[col("visa_type")]) \
.drop('sas_date')

In [3]:
immigration_fact.show(5)

NameError: name 'immigration_fact' is not defined

In [211]:
person_dim = immigration_staging.join(country_code_mapping_staging, 
                         immigration_staging["I94CIT"] == country_code_mapping_staging["code"], 
                         "left").withColumnRenamed("country", "country_of_origin").drop('code')
person_dim = person_dim.join(country_code_mapping_staging, 
                         person_dim["I94RES"] == country_code_mapping_staging["code"], 
                         "left").withColumnRenamed("country", "country_of_residence").drop('code')
person_dim = person_dim.select(col("cicid").alias("id").cast("int"),
                                            col("GENDER").alias("gender"),
                                            col("I94BIR").alias("age").cast("int"),
                                            col("country_of_origin").alias("country_of_origin"),
                                            col("country_of_residence").alias("country_of_residence"),
                                            col("BIRYEAR").alias("year_of_birth").cast("int"))

In [212]:
person_dim.show(5)

+-------+------+---+--------------------+--------------------+-------------+
|     id|gender|age|   country_of_origin|country_of_residence|year_of_birth|
+-------+------+---+--------------------+--------------------+-------------+
|5451672|     M| 52|'UNITED KINGDOM' ...|'UNITED KINGDOM' ...|         1964|
|6096389|     M| 35|'MEXICO Air Sea, ...|'MEXICO Air Sea, ...|         1981|
|3686942|     M| 63|                null|  'GERMANY'         |         1953|
|1579708|     M| 51|'NETHERLANDS'    ...|'NETHERLANDS'    ...|         1965|
| 829755|     M| 70|  'ROMANIA'         |  'ROMANIA'         |         1946|
+-------+------+---+--------------------+--------------------+-------------+
only showing top 5 rows



In [213]:
# Demographics dimension
demographics_dim = demographics_staging.withColumn("id", monotonically_increasing_id()) \
                    .select(col("id"),
                            col("State").alias("state"),
                            col("Median Age").alias("median_age"),
                            col("Female Population").alias("female_population"),
                            col("Number of Veterans").alias("number_of_veterans"),
                            col("Foreign-born").alias("foreign_born"),
                            col("Average Household Size").alias("average_household_size"),
                            col("State Code").alias("state_code"),
                            col("Race").alias("race"),
                            col("Count").alias("count"))


In [214]:
demographics_dim.show(5)

+---+------------+----------+-----------------+------------------+------------+----------------------+----------+--------------------+-----+
| id|       state|median_age|female_population|number_of_veterans|foreign_born|average_household_size|state_code|                race|count|
+---+------------+----------+-----------------+------------------+------------+----------------------+----------+--------------------+-----+
|  0|      Kansas|      34.6|           197601|             23978|       40270|                  2.56|        KS|American Indian a...| 8791|
|  1|Pennsylvania|      33.5|            59581|              5691|       19652|                  2.67|        PA|Black or African-...|22304|
|  2| Connecticut|      37.3|            41227|              3752|       25675|                  2.74|        CT|Black or African-...| 8454|
|  3|   Tennessee|      34.1|           340365|             27942|       88193|                  2.39|        TN|  Hispanic or Latino|67526|
|  4| Connect

In [215]:
demographics_dim.printSchema()

root
 |-- id: long (nullable = false)
 |-- state: string (nullable = true)
 |-- median_age: double (nullable = true)
 |-- female_population: integer (nullable = true)
 |-- number_of_veterans: integer (nullable = true)
 |-- foreign_born: integer (nullable = true)
 |-- average_household_size: double (nullable = true)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- count: integer (nullable = true)



In [225]:
weather_dim = weather_staging.withColumn("id", monotonically_increasing_id()) \
                              .select(col("id"),
                                     col("dt").alias("date"),
                                     col("AverageTemperature").alias("average_temperature"),
                                     col("AverageTemperatureUncertainty").alias("average_temperature_uncertainty"),
                                     col("State").alias("state"),
                                     col("Country").alias("country"))

In [226]:
weather_dim.show(5)

+---+----------+-------------------+-------------------------------+-----+-------+
| id|      date|average_temperature|average_temperature_uncertainty|state|country|
+---+----------+-------------------+-------------------------------+-----+-------+
|  0|1855-05-01|             25.544|                          1.171| Acre| Brazil|
|  1|1855-06-01|             24.228|                          1.103| Acre| Brazil|
|  2|1855-07-01|             24.371|                          1.044| Acre| Brazil|
|  3|1855-08-01|             25.427|                          1.073| Acre| Brazil|
|  4|1855-09-01|             25.675|                          1.014| Acre| Brazil|
+---+----------+-------------------+-------------------------------+-----+-------+
only showing top 5 rows



In [ ]:
airport_dim = airport_staging

In [227]:
airport_staging.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [228]:
# airport_staging.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.